In [1]:
from collections import namedtuple
Point = namedtuple("Point", "x y")

p = 9631668579539701602760432524602953084395033948174466686285759025897298205383
F = GF(p)

gx = 5664314881801362353989790109530444623032842167510027140490832957430741393367
gy = 3735011281298930501441332016708219762942193860515094934964869027614672869355
G = Point(gx, gy)

In [2]:
A = Point(x=3829488417236560785272607696709023677752676859512573328792921651640651429215, y=7947434117984861166834877190207950006170738405923358235762824894524937052000)
B = Point(x=9587224500151531060103223864145463144550060225196219072827570145340119297428, y=2527809441042103520997737454058469252175392602635610992457770946515371529908)
enc = bytes.fromhex('1536c5b019bd24ddf9fc50de28828f727190ff121b709a6c63c4f823ec31780ad30d219f07a8c419c7afcdce900b6e89b37b18b6daede22e5445eb98f3ca2e40')

In [3]:
f = lambda P: F(P.y^2 - P.x^3)
a = (f(A) - f(B)) * pow(A.x - B.x, -1, p)
b = f(A) - a * A.x

a = int(a)
b = int(b)
print('a =', a)
print('b =', b)
assert (4*a^3 + 27*b^2) % p == 0

a = 9605275265879631008726467740646537125692167794341640822702313056611938432994
b = 7839838607707494463758049830515369383778931948114955676985180993569200375480


\begin{align}
y^2 &= x^3 + ax + b \\
&= (x-\alpha)^2(x-\beta) \\
&= x^3 - (2\alpha+\beta)x^2 + (\alpha^2+2\alpha\beta)x - (\alpha^2\beta) \\
\\
&\Rightarrow
\begin{cases}
    2\alpha+\beta = 0 \\
    \alpha^2+2\alpha\beta = a \\
    -\alpha^2\beta = b \\
\end{cases}
\\
&\Rightarrow
\beta = -2\alpha
\\
&\Rightarrow
\begin{cases}
    \beta = -2\alpha \\
    2\alpha^3 = b \\
    -3\alpha^2 = a \\
\end{cases}\\
&\Rightarrow
\begin{cases}
    \alpha = \displaystyle\frac{b/2}{a/-3} \\
    \beta = -2\alpha \\
\end{cases}\\
\end{align}

In [4]:
alpha = (F(b)/2)/(F(a)/-3)
beta = -2*alpha

In [5]:
PF.<x> = PolynomialRing(F)
assert (x-PF(alpha))^2*(x-PF(beta)) == x^3 + PF(a)*x + PF(b)

In [6]:
def phi(P):
    k = sqrt(alpha-beta) * (P.x-alpha)
    return (P.y+k)/(P.y-k)

phi(G)

3595609028560051069710646173649895336250763792764650672569352125353978457253

$ \phi(A) = \phi(dA\cdot G) = \phi(G)^{dA} $

In [7]:
from sage.groups.generic import bsgs
def pohlig_hellman(p,g,h,fac = None, debug = False):
    F = IntegerModRing(p)
    g = F(g)
    h = F(h)
    if not fac:
        fac = set(ecm.factor(p-1))
    if debug: print(fac)
    xi = []
    pi = []
    for q in fac:
        cnt = 1
        while (p-1)%(q**(cnt+1)) == 0:
            cnt += 1
        if debug: print(q,cnt)
        k = q**cnt
        G = g**((p-1)//k)
        H = h**((p-1)//k)
        X = bsgs(G, H, (0, q))
        xi.append(X)
        pi.append(k)
        if debug: print(G, H, X)
    r = crt(xi, pi)
    return r

In [8]:
O = 'INFINITY'

def is_on_curve(P):
    if P == O:
        return True
    else:
        return (P.y**2 - (P.x**3 + a*P.x + b)) % p == 0 and 0 <= P.x and P.x < p and 0 <= P.y and P.y < p

def point_inverse(P):
    if P == O:
        return P
    return Point(P.x, -P.y % p)

inverse = lambda x, p: int(pow(x,-1,p))

def point_addition(P, Q):
    if P == O:
        return Q
    elif Q == O:
        return P
    elif Q == point_inverse(P):
        return O
    else:
        if P == Q:
            s = (3*P.x**2 + a)*inverse_mod(2*P.y, p) % p
        else:
            s = (Q.y - P.y) * inverse_mod((Q.x - P.x), p) % p
    Rx = (s**2 - P.x - Q.x) % p
    Ry = (s*(P.x - Rx) - P.y) % p
    R = Point(Rx, Ry)
    assert is_on_curve(R)
    return R

def point_multiply(P, d):
    bits = bin(d)[2:]
    Q = O
    for bit in bits:
        Q = point_addition(Q, Q)
        if bit == '1':
            Q = point_addition(Q, P)
    assert is_on_curve(Q)
    return Q

In [9]:
dA = pohlig_hellman(p,phi(G),phi(A))
assert A == point_multiply(G,dA)

In [10]:
import hashlib
k = point_multiply(B, dA).x
k = hashlib.sha512(str(k).encode('ascii')).digest()
FLAG = bytes(ci ^^ ki for ci, ki in zip(enc, k)).strip(b'\0').decode().strip()
print(FLAG)

FLAG{adbffefdb46a99fad0042dd3c10fdc414fadd25c}
